In [175]:
using Optim

In [176]:
function interpolate1D(vX, vY, x)
    imin = sum(vX.<x);
    imax = sum(vX.<x)+1;
    y = vY[imin]+( x-vX[imin] )/( vX[imax]-vX[imin] )*(vY[imax]-vY[imin]);
    return y
end

interpolate1D (generic function with 1 method)

In [177]:
# Value function
function valfun(v0, pBeta, pDelta, pAlpha, vK, k0, s, k)

    # This program gets the value function for a neoclassical growth model with
    # no uncertainty and CRRA utility
    
    g = interpolate1D(vK,v0,k); # smooths out previous value function
    
    c = k0^pAlpha-k + (1-pDelta)*k0; # consumption
    if c <= 0
        val =-888888888888888888-800*abs(c); # keeps it from going negative
    else
        val=(1/(1-s))*(c^(1-s)-1) + pBeta*g;
    end
    val =-val; # make it negative since we're maximizing and code is to minimize.
    
    return val
                
end

valfun (generic function with 1 method)

In [202]:
# Deterministic Growth model
# Original code by Eric Sims
# Tarik OCAKTAN - Luxembourg - November 2020

# State space dimension = 1

# Notation:
#  pName : structural parameters
#  iName : total number of variables related to the algorithm
#  idxName : indexation used in loops, i.e. idxName = 1,...,iName
#  dName : integer of a specific value of a variable, i.e. dNamess=steady state
#          of
#               variable 'Name'
#  vName : vector containing values of variable 'Name'
#  mName : matrix containing values of a variable 'Name'



# Structural parameters
pAlpha  =  .33;
pBeta   =  .95;
pDelta  = 0.1;
s       = 2;

# Algorithm parameters
 iIter = 1000; # maximum number of iterations
 iGridPoints = 4;
 iToler	= 0.01;  # convergence criterion

# Steady state value
 dKss = (pAlpha/(1/pBeta-(1-pDelta)))^(1/(1-pAlpha));
 dCss = dKss^(pAlpha)-pDelta*dKss;
 dIss = pDelta*dKss;
 dYss = dKss^pAlpha;

# Bounds of state space
 dKmin = dKss * 0.25;
 dKmax = dKss * 1.75;



In [203]:
# Construct a grid
 iStep = (dKmax-dKmin) / (iGridPoints-1);
 vK = collect(dKmin:iStep:dKmax);
 iGridPoints = size(vK,1); # overwriting iGridPoints in order to take into
                           # account additional point

# Storage space
 polfun = zeros(iGridPoints);
 v0 = zeros(iGridPoints);
 v1 = collect( zeros(iGridPoints) );
 k11 = collect( zeros(iGridPoints) );

 iDif = 10;
 idxIter = 0;

In [212]:


#while iDif>iToler && idxIter<iIter;
for idxIter=1:100
   for i=1:iGridPoints

      k0 = vK[i];
      opt = optimize( k -> valfun(v0, pBeta, pDelta, pAlpha, vK, k0, s, k),
                           dKmin, dKmax, GoldenSection());  

      k1 = Optim.minimizer(opt);  
        
      v1[i] = -valfun(v0, pBeta, pDelta, pAlpha, vK, k0, s, k1);
      k11[i] = k1;           
        
   end;
    
   iDif = sqrt( sum( v1-v0 ) );
   v0 = v1;
  
 
    
#   idxIter = idxIter+1;
end     
    
#end

